In [1]:
# packages

!pip install transformers 

import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
import pandas as pd
import pickle
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 85.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.7 MB/s eta 0:00:00


In [2]:
# get data - MUStARD
url = "https://raw.githubusercontent.com/soujanyaporia/MUStARD/master/data/sarcasm_data.json"
df = pd.read_json(url,orient='index')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# first time: feature extraction
# https://towardsdatascience.com/feature-extraction-with-bert-for-text-classification-533dde44dc2f

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModel.from_pretrained("distilbert-base-uncased").to(device)

tokenized_utterance = tokenizer(df["utterance"].values.tolist(), padding = True, truncation = True, return_tensors="pt")
tokenized_utterance = {k:torch.tensor(v).to(device) for k,v in tokenized_utterance.items()}
with torch.no_grad():
  hidden_utterance = model(**tokenized_utterance)
cls_utterance = hidden_utterance.last_hidden_state[:,0,:]
print("fin utterance")
file = open('drive/MyDrive/proj/bert_utterance', 'wb')
pickle.dump(cls_utterance, file)
file.close()

df["context"] = df.apply(lambda r:" ".join(r["context"]), axis=1)
tokenized_context = tokenizer(df["context"].values.tolist(), padding = True, truncation = True, return_tensors="pt")
tokenized_context = {k:torch.tensor(v).to(device) for k,v in tokenized_context.items()}
with torch.no_grad():
  hidden_context = model(**tokenized_context)
cls_context = hidden_context.last_hidden_state[:,0,:]
print("fin context")
file = open('drive/MyDrive/proj/bert_context', 'wb')
pickle.dump(cls_context, file)
file.close()


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
<ipython-input-4-7f337b79cefc>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokenized_utterance = {k:torch.tensor

fin utterance


<ipython-input-4-7f337b79cefc>:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokenized_context = {k:torch.tensor(v).to(device) for k,v in tokenized_context.items()}


fin context


In [5]:
# second time: load features
file = open('drive/MyDrive/proj/bert_utterance', 'rb')
cls_utterance = pickle.load(file)
file.close()
file2 = open('drive/MyDrive/proj/bert_context', 'rb')
cls_context = pickle.load(file2)
file2.close()

In [6]:
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from numpy import mean
from numpy import std


In [7]:
## SPLIT DATA

output_labels = df["sarcasm"].astype(int)

# Split data into train, validation and test sets
# First, split to train and test sets
# split
X_train, X_test, y_train, y_test = train_test_split(np.concatenate([cls_utterance.cpu(), cls_context.cpu()], axis=1), df["sarcasm"], test_size=0.3, random_state = 42)
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Normalize input features
scaler = MinMaxScaler()
X_train_norm = scaler.fit_transform(X_train)
#X_val_norm = scaler.transform(X_val)
X_test_norm = scaler.transform(X_test)

In [8]:
## FIND THE BEST PARAMETERS FOR LOGISTIC REGRESSION CLASSIFIER

# Define your model
model = LogisticRegression()

# Define the parameter grid to search over
param_grid = {'C': [0.1, 1.0, 3.0, 4.0, 5.0, 6.0, 10.0, 100.0 ],
               'penalty': ['l1', 'l2'],
                 'solver': ["liblinear", "sag", "saga", "newton-cg", "lbfgs"]}

# Perform a grid search to find the best parameters for your model
# NOTE: when using GridSearchCV, there is no need for a separate validation set.
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1')
grid_search.fit(X_train_norm, y_train)

# Print the best parameters and the best score
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means 

Best parameters: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
Best score: 0.6895081470396162


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
120 fits failed out of a total of 400.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceba

In [9]:
## EVALUATE ON TEST SET 

# Define your model with the best parameters
# C=1.0 works as well
#Best parameters: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}

model = LogisticRegression(C=0.1, penalty='l2', solver='liblinear')

# Train your model on the entire training set
model.fit(X_train_norm, y_train)

# Make predictions on new data
y_pred = model.predict(X_test_norm)

reportDict = classification_report(y_test, y_pred, output_dict=True)
f1_score = reportDict['weighted avg']['f1-score']
print("TEST F1-score:", f1_score)

print(classification_report(y_test, y_pred))#, output_dict=True, digits=3))
print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred))

TEST F1-score: 0.6625322113953945
              precision    recall  f1-score   support

       False       0.62      0.67      0.64        94
        True       0.70      0.65      0.68       113

    accuracy                           0.66       207
   macro avg       0.66      0.66      0.66       207
weighted avg       0.67      0.66      0.66       207

Confusion Matrix: 
 [[63 31]
 [39 74]]


### Bagging

In [16]:
from sklearn.ensemble import BaggingClassifier

# {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
# (C=3.0, penalty='l1', solver='saga')
# LogisticRegression(C=3.0, penalty='l1', solver='saga'

param_grid = {
    'base_estimator': [LogisticRegression(C=0.1, penalty='l2', solver='liblinear')],
    'n_estimators': [10, 50, 100],
    'max_samples': [0.5, 0.8, 1.0],
    'max_features': [0.5, 0.8, 1.0],
    'random_state' : [42],
}

bagging_clf = BaggingClassifier()
grid_search_bagging = GridSearchCV(bagging_clf, param_grid, cv=5, scoring='f1')
grid_search_bagging.fit(X_train_norm, y_train)

# Get the best parameters and the best Bagging model
best_params_bagging = grid_search_bagging.best_params_
#best_bagging = grid_search_bagging.best_estimator_

print(f'The best estimator is {best_params_bagging}')

""""
f1_scores = cross_val_score(model, X_val_norm, y_val, cv=5, scoring='f1')
print('CV F1 score: %.3f (%.3f)' % (mean(f1_scores), std(f1_scores)))
reportDict = classification_report(y_val, y_pred, output_dict=True)
f1_score = reportDict['weighted avg']['f1-score']
print("F1-score:", f1_score)
print(classification_report(y_val, y_pred))#, output_dict=True, digits=3))
print("Confusion Matrix: \n", confusion_matrix(y_val, y_pred))
"""

# test and evaluation
test_y_pred = grid_search_bagging.predict(X_test_norm)

reportDict = classification_report(y_test, test_y_pred, output_dict=True)
f1_score = reportDict['weighted avg']['f1-score']
print("TEST F1-score:", f1_score)
print(classification_report(y_test, test_y_pred))#, output_dict=True, digits=3))
print("Confusion Matrix: \n", confusion_matrix(y_test, test_y_pred))

/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/ensemble

The best estimator is {'base_estimator': LogisticRegression(C=0.1, solver='liblinear'), 'max_features': 0.5, 'max_samples': 0.5, 'n_estimators': 10, 'random_state': 42}
TEST F1-score: 0.6914580218472176
              precision    recall  f1-score   support

       False       0.65      0.70      0.67        94
        True       0.73      0.68      0.71       113

    accuracy                           0.69       207
   macro avg       0.69      0.69      0.69       207
weighted avg       0.69      0.69      0.69       207

Confusion Matrix: 
 [[66 28]
 [36 77]]


### Boosting

In [17]:
from sklearn.ensemble import AdaBoostClassifier

# {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
# (C=3.0, penalty='l1', solver='saga')

param_grid = {
    'base_estimator': [LogisticRegression(C=0.1, penalty='l2', solver='liblinear'),
                       LogisticRegression(C=3.0, penalty='l1', solver='saga')],
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 1],
    'random_state' : [42],
}

adaboost_clf = AdaBoostClassifier()
grid_search_adaboost = GridSearchCV(adaboost_clf, param_grid, cv=5, scoring='f1')
grid_search_adaboost.fit(X_train_norm, y_train)

# Get the best parameters and the best AdaBoost model
best_params_adaboost = grid_search_adaboost.best_params_
#best_adaboost = grid_search_adaboost.best_estimator_

print(f'The best estimator is {best_params_adaboost}')

"""
# validation and results
y_pred = adaboost.predict(X_val_norm)

f1_scores = cross_val_score(model, X_val_norm, y_val, cv=5, scoring='f1')
print('F1 score: %.3f (%.3f)' % (mean(f1_scores), std(f1_scores)))

reportDict = classification_report(y_val, y_pred, output_dict=True)
f1_score = reportDict['weighted avg']['f1-score']
print("F1-score:", f1_score)
print(classification_report(y_val, y_pred))#, output_dict=True, digits=3))
print("Confusion Matrix: \n", confusion_matrix(y_val, y_pred))
"""


# test and evaluation
test_y_pred = grid_search_adaboost.predict(X_test_norm)

reportDict = classification_report(y_test, test_y_pred, output_dict=True)
f1_score = reportDict['weighted avg']['f1-score']
print("TEST F1-score:", f1_score)
print(classification_report(y_test, test_y_pred))#, output_dict=True, digits=3))
print("Confusion Matrix: \n", confusion_matrix(y_test, test_y_pred))

/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/ensemble

The best estimator is {'base_estimator': LogisticRegression(C=0.1, solver='liblinear'), 'learning_rate': 1, 'n_estimators': 150, 'random_state': 42}
TEST F1-score: 0.6720979404081152
              precision    recall  f1-score   support

       False       0.63      0.67      0.65        94
        True       0.71      0.67      0.69       113

    accuracy                           0.67       207
   macro avg       0.67      0.67      0.67       207
weighted avg       0.67      0.67      0.67       207

Confusion Matrix: 
 [[63 31]
 [37 76]]


### Stacking

In [18]:
from sklearn.ensemble import StackingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

base_estimators = [
    ('svc', SVC(degree=10, kernel="poly")),
    ('dt', DecisionTreeClassifier()),
    ('lr', LogisticRegression(C=0.1, penalty="l2", solver="liblinear"))
]

stacking_clf = StackingClassifier(estimators=base_estimators, final_estimator=LogisticRegression())

param_grid = {
    'dt__max_depth': [1, 2, 3],
    'final_estimator__C': [0.1, 1, 10]
}

grid_search_stacking = GridSearchCV(stacking_clf, param_grid, cv=5, scoring='f1')
grid_search_stacking.fit(X_train_norm, y_train)


"""
meta_model = LogisticRegression()
stacking = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5)
stacking.fit(X_train_norm, y_train)
y_pred = stacking.predict(X_val_norm)

f1_scores = cross_val_score(model, X_val_norm, y_val, cv=5, scoring='f1')
print('CV F1 score: %.3f (%.3f)' % (mean(f1_scores), std(f1_scores)))
reportDict = classification_report(y_val, y_pred, output_dict=True)
f1_score = reportDict['weighted avg']['f1-score']
print("F1-score:", f1_score)
print(classification_report(y_val, y_pred))#, output_dict=True, digits=3))
print("Confusion Matrix: \n", confusion_matrix(y_val, y_pred))
"""

# Get the best parameters and the best Stacking model
best_params_stacking = grid_search_stacking.best_params_
#best_stacking = grid_search_stacking.best_estimator_

print(f'The best estimator is {best_params_stacking}')

# Make predictions using the best model
test_y_pred = grid_search_stacking.predict(X_test_norm)

reportDict = classification_report(y_test, test_y_pred, output_dict=True)
f1_score = reportDict['weighted avg']['f1-score']
print("TEST F1-score:", f1_score)
print(classification_report(y_test, test_y_pred))#, output_dict=True, digits=3))
print("Confusion Matrix: \n", confusion_matrix(y_test, test_y_pred))

The best estimator is {'dt__max_depth': 2, 'final_estimator__C': 10}
TEST F1-score: 0.6576794965121349
              precision    recall  f1-score   support

       False       0.61      0.66      0.64        94
        True       0.70      0.65      0.68       113

    accuracy                           0.66       207
   macro avg       0.66      0.66      0.66       207
weighted avg       0.66      0.66      0.66       207

Confusion Matrix: 
 [[62 32]
 [39 74]]
